# Exercise 3.1

To write a likelihood function for the locomotive problem, we had to answer this question: “If the railroad has N locomotives, what is the probability that we see number 60?”

The answer depends on what sampling process we use when we observe the locomotive. In this chapter, I resolved the ambiguity by specifying that there is only one train-operating company (or only one that we care about).

But suppose instead that there are many companies with different numbers of trains. And suppose that you are equally likely to see any train operated by any company. In that case, the likelihood function is different because you are more likely to see a train operated by a large company.

As an exercise, implement the likelihood function for this variation of the locomotive problem, and compare the results.

----

Initially we had to estimate the number of locomotives the railroad had ($N$) based on the fact that we observed one with the number 60 ($n=60$)
$$P(N|n=60)=\frac{P(n=60|N)P(N)}{P(n=60)}$$

The entire fleet was composed of only one company. So there were locomotives numbered 1 to $N$. But now we have many company, so $N$ is the sum of all companies. Each company $C_i$ has $N_i$ locomotives. We get a fleet composed of the numbers $1,\ldots,N_1,1,\ldots,N_2,1,\ldots,N_m$.

Since every company has at least 1 locomotive, there are $m$ locomotives numbered 1. For the rest it depends on the number of locomotives each company has.

Let's sum up a few details that we have

- We have M companies labeled $C_i$ and each of them has $N_i$ locomotives.
- Each train, no matter the company, has the same probability of being observed, $\frac{1}{N}$ where $N=\sum_{j=1}^m{N_j}$. 
- Once we know the company, each locomotive has a probability of $\frac{1}{N_i}$ of being observed.
- Each company has a probability of $\frac{N_i}{N_{tot}}$ of being picked

We are tasked with finding the new likelihood $P(n=60|N)$, or equivalently $P(n=60|N_1,\ldots,N_m)$ because there are many company. For example, the probability of observing a locomotive numbered 1 if we have $m$ companies is $\frac{m}{N}$

### Theory

What is the probability that we observe a locomotive with the number n, $P(N=n|C_m)$?

Where $C_m$ is the distribution of companies and their respective sizes.

The problem is very similar to finding the probability of picking a vanilla cookie in chapter 2. We used the law of total probabilities then, we will use it again here.

$P(N=n|C_m)=\sum_{i=1}^m{P(N=n,C_i)}=\sum_{i=1}^m{P(N=n|C_i)P(C_i)}=
\sum_{i=1}^m{\frac{I_i}{N_i}\frac{N_i}{N_{tot}}}=
\frac{\sum_{i=1}^m{I_i}}{N_{tot}}$

Where

$I_i=\begin{cases}
    0,& n < N_i\\
    1,              & \text{otherwise}
\end{cases}
$

In other words, seeing a locomotive with the number $n$ is only as likely as there are number of locomotives with that number across all companies.

### Practice

In [2]:
from code.train3 import Train2

In [ ]:
class Train3(Train2):
    """
    Keep the same prior as Train2 which initializes 
    the hypotheses with a power law distribution
    """
    
    def Likelihood(self, data, hypo):
        """
        
        """
        if hypo < data:
            return 0
        else:
            return 1.0/hypo

In [ ]:
# %load code/train3.py
"""This file contains code for use with "Think Bayes",
by Allen B. Downey, available from greenteapress.com

Copyright 2012 Allen B. Downey
License: GNU GPLv3 http://www.gnu.org/licenses/gpl.html
"""

import thinkbayes
import thinkplot

from thinkbayes import Pmf, Percentile
from dice import Dice


class Train(Dice):
    """Represents hypotheses about how many trains the company has."""


class Train2(Dice):
    """Represents hypotheses about how many trains the company has."""

    def __init__(self, hypos, alpha=1.0):
        """Initializes the hypotheses with a power law distribution.

        hypos: sequence of hypotheses
        alpha: parameter of the power law prior
        """
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, hypo**(-alpha))
        self.Normalize()


def MakePosterior(high, dataset, constructor):
    """Makes and updates a Suite.

    high: upper bound on the range of hypotheses
    dataset: observed data to use for the update
    constructor: function that makes a new suite

    Returns: posterior Suite
    """
    hypos = xrange(1, high+1)
    suite = constructor(hypos)
    suite.name = str(high)

    for data in dataset:
        suite.Update(data)

    return suite


def ComparePriors():
    """Runs the analysis with two different priors and compares them."""
    dataset = [60]
    high = 1000

    thinkplot.Clf()
    thinkplot.PrePlot(num=2)

    constructors = [Train, Train2]
    labels = ['uniform', 'power law']

    for constructor, label in zip(constructors, labels):
        suite = MakePosterior(high, dataset, constructor)
        suite.name = label
        thinkplot.Pmf(suite)

    thinkplot.Save(root='train4',
                xlabel='Number of trains',
                ylabel='Probability')

def main():
    ComparePriors()

    dataset = [30, 60, 90]

    thinkplot.Clf()
    thinkplot.PrePlot(num=3)

    for high in [500, 1000, 2000]:
        suite = MakePosterior(high, dataset, Train2)
        print high, suite.Mean()

    thinkplot.Save(root='train3',
                   xlabel='Number of trains',
                   ylabel='Probability')

    interval = Percentile(suite, 5), Percentile(suite, 95)
    print interval

    cdf = thinkbayes.MakeCdfFromPmf(suite)
    interval = cdf.Percentile(5), cdf.Percentile(95)
    print interval


if __name__ == '__main__':
    main()
